In [25]:
import os
import requests
import json 
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
from pandas.tseries.offsets import DateOffset
from pandas import json_normalize
import time
from defillama import DefiLlama
from datetime import datetime
llama = DefiLlama()
import datetime as dt
import pytz
import sqlite3
from datetime import datetime, timezone

# flows = 2.5 seconds
# stables = half a min to a minute and a half
# tvl = 1 second
# dex vol = 

In [39]:
def get_most_recent_net_flows(chain_name: str):
    ids = []
    aggregated_data = {}
    
    # Fetch bridge data
    bridges_url = "https://bridges.llama.fi/bridges?includeChains=true"
    bridges_response = requests.get(bridges_url).json()
    
    # Collect IDs for bridges that involve the specified chain
    for bridge in bridges_response['bridges']:
        if chain_name in bridge['chains']:
            ids.append(bridge['id'])
    
    # Initialize variables to track the most recent date and its net flows
    most_recent_date = None
    most_recent_net_flows = 0
    
    for id in ids:
        url = f"https://bridges.llama.fi/bridgevolume/{chain_name}?id={id}"
        vol_data = requests.get(url).json()[-60:]  # Retrieve the last 60 entries
        
        for entry in vol_data:
            # Convert entry date to a datetime object in 'US/Eastern' timezone
            entry_date = datetime.utcfromtimestamp(int(entry['date'])).replace(tzinfo=pytz.utc).astimezone(pytz.timezone("US/Eastern"))
            entry_date_str = entry_date.strftime('%Y-%m-%d')
            
            # Update the most recent date and its net flows if this entry is more recent
            if most_recent_date is None or entry_date > most_recent_date:
                most_recent_date = entry_date
                most_recent_net_flows = entry['depositUSD'] - entry['withdrawUSD']
            elif entry_date == most_recent_date:
                # If the entry date matches the most recent date, aggregate the net flows
                most_recent_net_flows += entry['depositUSD'] - entry['withdrawUSD']
    
    # Return the most recent date and its aggregated net flows
    if most_recent_date:
        aggregated_data[most_recent_date.strftime('%Y-%m-%d')] = round(most_recent_net_flows,2)
    return aggregated_data

In [46]:
def get_most_recent_stables(chain_name: str):
    """
    Fetches the most recent stablecoin total value for the specified chain.
    """
    stable_ids = list(range(1, 161))  # Assuming stablecoin IDs from 1 to 160
    most_recent_date = None
    most_recent_total_stables = 0
    
    for id in stable_ids:
        url = f"https://stablecoins.llama.fi/stablecoincharts/{chain_name}?stablecoin={id}"
        response = requests.get(url)
        if response.status_code == 200:
            stables_data = response.json()
            if stables_data:
                # Assuming the last entry is the most recent
                entry = stables_data[-1]
                date = datetime.utcfromtimestamp(int(entry['date'])).replace(tzinfo=pytz.utc).astimezone(pytz.timezone("US/Eastern"))
                
                # Check if this entry's date is the most recent we've encountered
                if most_recent_date is None or date > most_recent_date:
                    most_recent_date = date
                    most_recent_total_stables = entry.get('totalCirculating', {}).get('peggedUSD', 0)
                elif date == most_recent_date:
                    # If the entry's date matches the most recent date, aggregate the totals
                    most_recent_total_stables += entry.get('totalCirculating', {}).get('peggedUSD', 0)
    
    # Return the most recent date and its aggregated total stablecoin value
    if most_recent_date:
        return {most_recent_date.strftime('%Y-%m-%d'): most_recent_total_stables}
    else:
        return {}

In [50]:
def get_most_recent_tvl(chain_name: str):
    """
    Fetches the most recent Total Value Locked (TVL) for the specified chain.
    """
    tvl_url = f"https://api.llama.fi/v2/historicalChainTvl/{chain_name}"
    response = requests.get(tvl_url)
    if response.status_code == 200:
        tvl_data = response.json()
        if tvl_data:
            # Assuming the last entry is the most recent
            most_recent_entry = tvl_data[-1]
            date = datetime.utcfromtimestamp(int(most_recent_entry['date'])).replace(tzinfo=pytz.utc).astimezone(pytz.timezone("US/Eastern"))
            most_recent_date = date.strftime('%Y-%m-%d')
            most_recent_tvl = most_recent_entry['tvl']
            
            # Return the most recent date and its TVL
            return {most_recent_date: most_recent_tvl}
    return {}

In [52]:
def get_dex_vol(chain_name:str):
    """
    Get aggregate dex volume done on each chain aggregated across all dexes
    """
    dex_volume = {'24H': 0, '7D': 0, '30D': 0}
    dex_url = f"https://api.llama.fi/overview/dexs/{chain_name}?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"

    try:
        dex_data = requests.get(dex_url).json()
        for entry in dex_data['protocols']:
            dex_volume['24H'] += round(entry.get('total24h', 0),2) if entry.get('total24h') is  not None else 0 
            dex_volume['7D'] += round(entry.get('total7d', 0),2) if entry.get('total7d') is  not None else 0 
            dex_volume['30D'] += round(entry.get('total30d', 0),3) if entry.get('total30d') is  not None else 0 
    except Exception as e:
        print(f"An error occurred: {e}")

    return dex_volume


### Connect back to the SQL databse with historical data, and update the most recent dates data:

In [59]:
def entry_exists(chain_name, date, table_name):
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    
    query = f"SELECT EXISTS(SELECT 1 FROM {table_name} WHERE chain_name=? AND date=? LIMIT 1)"
    cursor.execute(query, (chain_name, date))
    exists = cursor.fetchone()[0]
    
    conn.close()
    return exists == 1

def insert_data(table_name, data_dict):
    """
    Insert data into the specified table.
    
    Parameters:
    - table_name: The name of the table to insert data into.
    - data_dict: A dictionary where keys are column names and values are the data to insert.
    """
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    
    # Prepare column names and placeholders for values
    columns = ', '.join(data_dict.keys())
    placeholders = ', '.join(['?' for _ in data_dict])
    
    query = f'''INSERT OR REPLACE INTO {table_name} ({columns}) VALUES ({placeholders})'''
    cursor.execute(query, list(data_dict.values()))
    
    conn.commit()
    conn.close()

In [60]:
def fetch_and_insert_flows(chain_name):
    # Use the modified get_net_flows function that focuses on the most recent data
    flows_data = get_most_recent_net_flows(chain_name)
    for date, net_inflow in flows_data.items():
        if not entry_exists(chain_name, date, 'net_flows'):
            insert_data('net_flows', {
                'chain_name': chain_name,
                'date': date,
                'net_inflow': net_inflow
            })

def fetch_and_insert_stablecoins(chain_name):
    # Use the modified get_stables function
    stables_data = get_most_recent_stables(chain_name)
    for date, total_stables in stables_data.items():
        if not entry_exists(chain_name, date, 'Stablecoins'):
            insert_data('Stablecoins', {
                'chain_name': chain_name,
                'date': date,
                'total_stables': total_stables
            })

def fetch_and_insert_tvl(chain_name):
    # Use the modified get_tvl function
    tvl_data = get_most_recent_tvl(chain_name)
    for date, tvl in tvl_data.items():
        if not entry_exists(chain_name, date, 'TVL'):
            insert_data('TVL', {
                'chain_name': chain_name,
                'date': date,
                'TVL': tvl
            })

def fetch_and_insert_dex_volume(chain_name):
    dex_volume_data = get_dex_vol(chain_name)  # Ensure this function returns the most recent 24H, 7D, and 30D volume data
    # Prepare the data dictionary for insertion
    formatted_dex_volume_data = {
        'chain_name': chain_name,
        'volume_24h': dex_volume_data.get('24H', 0),
        'volume_7d': dex_volume_data.get('7D', 0),
        'volume_30d': dex_volume_data.get('30D', 0),
    }
    # Directly insert or replace the DEX volume data for the chain
    insert_data('Dex_Volume', formatted_dex_volume_data)



In [61]:
chains = ['Ethereum', 'Solana', 'Base', 'Sei', 'Sui', 'Injective', 'Avalanche', 'Optimism']

for chain in chains:
    fetch_and_insert_flows(chain)
    fetch_and_insert_stablecoins(chain)
    fetch_and_insert_tvl(chain)
    fetch_and_insert_dex_volume(chain)


An error occurred: type NoneType doesn't define __round__ method
An error occurred: type NoneType doesn't define __round__ method


## TO DO:
- Calculate % changes and stuff by chain for each data point by pulling from SQL
- Find a way to present it pretty like slurpxbt
- Find a way to automate the data pull + addition to db and email sending